In [1]:
import dask
import dask_image.imread
import dask.distributed
import skimage.io as io

import numpy as np

from sklearn.svm import SVC
from sklearn import metrics

import os
import sys
current = os.path.dirname(os.path.realpath(os.getcwd()))
parent = os.path.dirname(current)
sys.path.append(parent)
from Emotions_Detection.Models.features import lpq, lpq_plus, PHOG_Algorithm

In [2]:
# Constants and Vaiables
IMG_TRAIN_PATH = "../Assets/data/train"
IMG_TEST_PATH = "../Assets/data/test"
HAPPY_PATH = "../Assets/data/train/happy/"

train_folders = os.listdir(IMG_TRAIN_PATH)  # Emotions Folders
test_folders = os.listdir(IMG_TEST_PATH)  # Emotions Folders

# Intialize dash board for DASK
# client = dask.distributed.Client()  # Create a local cluster  
cluster = dask.distributed.LocalCluster(n_workers=12, threads_per_worker=2)
client = dask.distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 15.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57994,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 15.91 GiB
Comm: tcp://127.0.0.1:58114,Total threads: 2
Dashboard: http://127.0.0.1:58117/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:58002,


### **Auxilary Functions:**

In [3]:
# Auxiary Function Calls
@dask.delayed
def read_img_parallel(img_path):
    return io.imread(img_path, as_gray=True)

# Reading A Directory of images folders
def read_data(path: str):
    x_img = []
    y_img = []
    tasks = []
    folders = os.listdir(path)
    for folder in folders:
        folder_path = os.path.join(path, folder)
        pattern = os.path.join(folder_path, 'im*.png')
        imgs = dask_image.imread.imread(pattern)
        tasks.append(imgs)
        # Set Labels While Initializing
        labels = [folder for _ in range(len(imgs))]
        y_img.extend(labels)
    
    print("Implementing Tasks...")
    for task in tasks:
        x_img.extend(task.compute())
        print("Done...")
    return x_img, y_img

def get_phog(images):
    phog = []
    for img in images:
        phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
        phog.append(phog_desc)

    print('Starting PHOG Set...')
    phog = dask.compute(phog)[0]
    print('Done PHOG Set...')
    return phog

def get_lpq(images):
    lpq_train = []
    for img in images:
        lpq_desc = dask.delayed(lpq)(img)
        lpq_train.append(lpq_desc)

    print('Starting LPQ Set...')
    lpq_train = dask.compute(lpq_train)[0]
    print('Done LPQ Set...')
    return lpq_train
        

# **The pipeline of Emotion Module**

### **1. Reading the dataset (training, test):**

In [4]:
x_img, y_train = read_data(IMG_TRAIN_PATH)
x_img_test, y_test = read_data(IMG_TEST_PATH)

Implementing Tasks...
Done...
Done...
Done...
Done...
Done...
Done...
Done...
Implementing Tasks...
Done...
Done...
Done...
Done...
Done...
Done...
Done...


In [5]:
phog_train_feat = get_phog(x_img)
phog_test_feat = get_phog(x_img_test)

Starting PHOG Set...
Done PHOG Set...
Starting PHOG Set...
Done PHOG Set...


In [6]:
x_train = np.array(phog_train_feat)
x_test = np.array(phog_test_feat)
x_train.shape, x_test.shape, (680)

((28709, 680), (7178, 680), 680)

In [9]:
#Create a svm Classifier
# SVC(C=1, gamma=1, kernel='poly')
clf = SVC(C=1, gamma=1)
print("classifier initialized...")

#Train the model using the training sets
clf.fit(x_train, y_train)
print("classifier Trained...")

#Predict the response for test dataset
y_pred = clf.predict(x_test)

classifier initialized...
classifier Trained...


In [10]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average=None))
print("Recall:",metrics.recall_score(y_test, y_pred, average=None))

Accuracy: 0.24714405126776262
Precision: [0.         0.         0.         0.24714405 0.         0.
 0.        ]
Recall: [0. 0. 0. 1. 0. 0. 0.]


d:\Grad. Project\GP2\Codes\repo_emotion\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
#Create a svm Classifier
clf = SVC(C=1, gamma=1, kernel='poly', degree=5)
print("classifier initialized...")

#Train the model using the training sets
clf.fit(x_train, y_train)
print("classifier Trained...")

#Predict the response for test dataset
y_pred = clf.predict(x_test)

classifier initialized...
classifier Trained...


In [8]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average=None))
print("Recall:",metrics.recall_score(y_test, y_pred, average=None))

Accuracy: 0.24714405126776262
Precision: [0.         0.         0.         0.24714405 0.         0.
 0.        ]
Recall: [0. 0. 0. 1. 0. 0. 0.]


d:\Grad. Project\GP2\Codes\repo_emotion\venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
#Create a svm Classifier
clf = SVC(C=1, gamma=1, kernel='sigmoid')
print("classifier initialized...")

#Train the model using the training sets
clf.fit(x_train, y_train)
print("classifier Trained...")

#Predict the response for test dataset
y_pred = clf.predict(x_test)

classifier initialized...
classifier Trained...


In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average=None))
print("Recall:",metrics.recall_score(y_test, y_pred, average=None))

Accuracy: 0.41195319030370575
Precision: [0.37611408 1.         0.3472     0.45454545 0.34863946 0.3554717
 0.52844501]
Recall: [0.22025052 0.04504505 0.21191406 0.69052988 0.3325223  0.3777065
 0.50300842]
